In [1]:
import numpy as np
import cv2

In [2]:
img = cv2.imread(r'yolo004.png')
img.shape

(518, 520, 3)

In [3]:
width = int(img.shape[1])
height = int(img.shape[0])

In [4]:
img =cv2.resize(img,(width,height))

In [5]:
net = cv2.dnn.readNet('yolov3-tiny.weights','yolov3-tiny.cfg')

In [6]:
classes = []
with open('coco.names.txt','r')as f:
    classes = f.read().splitlines()

In [7]:
# classes

In [8]:
blob = cv2.dnn.blobFromImage(img, 1/255, (416,416), (0,0,0), swapRB = False, crop = False)

In [9]:
# for each in blob:
#     for n, image in enumerate(each):
#         cv2.imshow(str(n), image)
#         cv2.waitKey(0)
#         cv2.destroyAllWindows()

In [10]:
net.setInput(blob)
output_layers = net.getUnconnectedOutLayersNames()

In [11]:
print(output_layers)

['yolo_16', 'yolo_23']


In [12]:
layerOutputs = net.forward(output_layers)

In [13]:
print(layerOutputs)

[array([[0.0597829 , 0.05273973, 0.17765619, ..., 0.        , 0.        ,
        0.        ],
       [0.03927024, 0.04452548, 0.2241416 , ..., 0.        , 0.        ,
        0.        ],
       [0.03310811, 0.0452447 , 0.96758646, ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.9462963 , 0.96211433, 0.1984945 , ..., 0.        , 0.        ,
        0.        ],
       [0.9559401 , 0.9531429 , 0.22084059, ..., 0.        , 0.        ,
        0.        ],
       [0.9669117 , 0.9533934 , 0.7730628 , ..., 0.        , 0.        ,
        0.        ]], dtype=float32), array([[0.02193795, 0.0198808 , 0.02341989, ..., 0.        , 0.        ,
        0.        ],
       [0.0217988 , 0.02391635, 0.03281626, ..., 0.        , 0.        ,
        0.        ],
       [0.01350834, 0.02137448, 0.1280345 , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.97700554, 0.9779301 , 0.02420612, ..., 0.        , 0.        ,
        0.        ],
       [0.9779004 

In [14]:
boxes = []
confidences = []
class_ids = []

for each in layerOutputs:
    for detection in each:
        scores = detection[5:]
        class_id = np.argmax(scores)
        confidence = scores[class_id]
        
        if confidence > 0.4 :
            center_x = int(detection[0] * width) 
            center_y = int(detection[1] * height)
            w = int(detection[2] * width)
            h = int(detection[3] * height)
            
            x = int(center_x - (w/2))
            y = int(center_y - (h/2))
            
            boxes.append([x,y,w,h])
            confidences.append(float(confidence))
            class_ids.append(class_id)
print(len(boxes))
            
indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.4,0.4)
print(indexes.flatten())

font = cv2.FONT_HERSHEY_PLAIN
colors = np.random.uniform(0,255,size = (len(boxes), 3))

# print(indexes.flatten())

for i in indexes.flatten():
    x,y,w,h = boxes[i]
    label = str(class_ids[i])
    j = int(label)
    confidence = str(round(confidences[i],2))
    color = colors[i]
    cv2.rectangle(img, (x,y), (x+w, y+h),color,2 )
    cv2.putText(img, classes[j]+" "+ confidence+'%', (x,y-20), font, 1,(255,255,255),2)


4
[0 2 3 1]


In [15]:
cv2.imshow("Yolo Image", img)
cv2.waitKey(0)
cv2.destroyAllWindows()